# **F-Algebra and F-Coalbgera**


In [1]:
using Pkg
Pkg.activate(".")
using MLStyle

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`


In [2]:
struct NilF end
struct ConsF{c,a}
    _1::c
    _2::a
end
ListF{c,a} = Union{NilF, ConsF{c,a}}
fmap(f::Function, x::ConsF) = ConsF(x._1,f(x._2))
fmap(f::Function, x::NilF)  = NilF()


# data List c = Nil | Cons c (List c) ----- In Haskell
struct Nil end
struct Cons{c}
    _1::c
    _2::Union{Nil, Cons{c}}
end

List{T} = Union{Nil, Cons{T}};

In [3]:
fix(::NilF)  = Nil()
fix(x::ConsF)= Cons(x._1,x._2)
fix(ConsF(10, Nil()))

Cons{Int64}(10, Nil())

Note that `ListF{I,T}` is a functor when we fix the first type. Fix `I` to be `Int`. Now we have
`ListF{Int,T}`, which we use to create the category of algebras over.
The (`List{Int}`,`fix`) algebra is the initial object in the `ListF{Int,_}`-Algebra category.
Let's define another F-Algebra for the functor `ListF{Int,_}`. For that, we need a *carrier* type `T`
and an evaluation function `eval(x::ListF{Int,T})::T`.

Take `T` to be `Int`. We can define the evaluation as the following:

In [4]:
evalSum(::NilF)   = 0
evalSum(x::ConsF{Int,Int}) = x._1 + x._2;

In [5]:
cataSum(::Nil)   = evalSum(NilF())
cataSum(x::Cons) = evalSum(ConsF(x._1,(cataSum(x._2))))
cataSum(Cons(10, Cons(20,Nil())))

30

Note that (`Int`, `evalSum`) is a `ListF{Int,_}`-Algebra. With the same carrier (`Int`) we can
define a new algebra by changing `evalSum` for `evalProd`.

Hence, we get a new catamorphism `cataProd`.

In [6]:
evalProd(::NilF)   = 1
evalProd(x::ConsF{Int,Int}) = x._1 * x._2;

cataProd(::Nil)   = evalProd(NilF())
cataProd(x::Cons) = evalProd(ConsF(x._1,(cataProd(x._2))))
cataProd(Cons(10, Cons(20,Nil())))

200

### Catamorphism

We can perceive that there is a general construction. Which is defined as

```Haskell
cata alg = alg . fmap (cata alg) . unFix
```

A theorem by Lambek states that for the initial F-Algebra, which is `(List{c}, fix)` in our case,
there is an inverse function to `fix`, which we call `unfix`. While
`fix(x::ListF{c, List{c}})::List{c}`, the inverse take a vaue of `List{c}` and returns
a value of type `ListF{c,List{c}}`. Let's implement this.

In [7]:
unfix(::Nil) = NilF()
unfix(x::Cons)= ConsF(x._1,x._2)
unfix(Cons(10, Nil()))

ConsF{Int64, Nil}(10, Nil())

We can now implement the generic catamorphism. Note that the idea is to
do a single implementation `cata` that takes an algebra (`eval`), instead
of implementing every single `cataSum`,  `cataProd` and so on.

In [8]:
x = Cons(20,Cons(10,Nil()))

cata(alg::Function, x::List) = alg(fmap(y -> cata(alg, y), unfix(x)))
@show cata(evalSum,Nil())
@show cata(evalSum,Cons(10,Nil()))
@show cata(evalSum,Cons(10,Cons(10,Nil())));

@show cata(evalProd,Nil())
@show cata(evalProd,Cons(10,Nil()))
@show cata(evalProd,Cons(10,Cons(10,Nil())));

cata(evalSum, Nil()) = 0
cata(evalSum, Cons(10, Nil())) = 10
cata(evalSum, Cons(10, Cons(10, Nil()))) = 20
cata(evalProd, Nil()) = 1
cata(evalProd, Cons(10, Nil())) = 10
cata(evalProd, Cons(10, Cons(10, Nil()))) = 100


**Example 2 - Natural Numbers Functor** 

In [28]:
@data NatF begin
    ZeroF()
    SuccF(_1)
end
fmap(f::Function, x::ZeroF)  = ZeroF()
fmap(f::Function, x::SuccF) = SuccF(f(x._1))

@data Nat begin
    Zero()
    Succ(Nat)
end
unfix(::Zero) = ZeroF()
unfix(x::Succ)= SuccF(x._1)
unfix(Succ(Succ(Zero())))

fib(::ZeroF) = (1,1)
fib(x::SuccF) = (x._1[2], x._1[1] + x._1[1])

@show fib(SuccF((10,10)))
@show fib(ZeroF());

cata(alg::Function, x::Nat) = alg(fmap(y -> cata(alg, y), unfix(x)))

@show cata(fib, Succ(Succ(Zero())));

fib(SuccF((10, 10))) = (10, 20)
fib(ZeroF()) = (1, 1)
cata(fib, Succ(Succ(Zero()))) = (2, 2)
